<a href="https://colab.research.google.com/github/prajeet26/Bank_credit_risk_analysis/blob/main/bank_risk_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics import f1_score, accuracy_score

appl = pd.read_csv('/content/drive/MyDrive/applicant.csv')
loan = pd.read_csv('/content/drive/MyDrive/loan.csv')

In [ ]:
appl.head()


In [ ]:
loan.head()

In [ ]:
appl.shape



In [ ]:
loan.shape

In [ ]:
appl.describe

In [ ]:
from tabulate import tabulate
print(tabulate(appl.head(), headers = 'keys', tablefmt = 'psql'))

In [ ]:
from tabulate import tabulate
print(tabulate(loan.head(), headers = 'keys', tablefmt = 'psql'))

In [ ]:
print(tabulate(appl.describe(), headers = 'keys', tablefmt = 'psql'))

In [ ]:
appl.dtypes

In [ ]:
appl.isnull().sum()

In [ ]:
loan.drop(["loan_application_id"], axis = 1, inplace=True)

In [ ]:
loan.sample(4)

In [ ]:
final=pd.merge(loan,appl,on="applicant_id")

In [ ]:
final.head()

In [ ]:
final.describe()

In [ ]:
final['Principal_loan_amount']=final['Principal_loan_amount'].div(100000).round(2)#FOR CONVINIENCE MADE AMOUNT INTO LAKHS

In [ ]:
final.describe()

In [ ]:
final.drop(["applicant_id"], axis = 1, inplace=True)

In [ ]:
pd.options.display.max_columns = None

In [ ]:
final.sample(1)

In [ ]:
pd.unique(final['Loan_history'])

In [ ]:
pd.unique(final['Employment_status'])

In [ ]:
#FREQUENCY COUNT
import matplotlib.ticker as ticker
ncount=len(final)
plt.figure(figsize=(12,8))
ax = sns.countplot(x="high_risk_applicant", data=final, order=[0,1])
ax2=ax.twinx()
ax2.yaxis.tick_left()
ax.yaxis.tick_right()
ax2.set_ylabel('Frequency [%]')
for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y),ha='center', va='bottom')

In [ ]:
#FREQUENCY COUNT
import matplotlib.ticker as ticker
ncount=len(final)
plt.figure(figsize=(25,8))
ax = sns.countplot(x="Loan_history", data=final, order=['critical/pending loans at other banks',
       'existing loans paid back duly till now',
       'delay in paying off loans in the past',
       'no loans taken/all loans paid back duly',
       'all loans at this bank paid back duly'])
ax2=ax.twinx()
ax2.yaxis.tick_left()
ax.yaxis.tick_right()
ax2.set_ylabel('Frequency [%]')
for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y),ha='center', va='bottom')
    

In [ ]:
#FREQUENCY COUNT
import matplotlib.ticker as ticker
ncount=len(final)
plt.figure(figsize=(25,8))
ax = sns.countplot(x="Primary_applicant_age_in_years", data=final)
ax2=ax.twinx()
ax2.yaxis.tick_left()
ax.yaxis.tick_right()
ax2.set_ylabel('Frequency [%]')
for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y),ha='center', va='bottom')

In [ ]:
#FREQUENCY COUNT
import matplotlib.ticker as ticker
ncount=len(final)
plt.figure(figsize=(25,8))
ax = sns.countplot(x="Number_of_existing_loans_at_this_bank", data=final)
ax2=ax.twinx()
ax2.yaxis.tick_left()
ax.yaxis.tick_right()
ax2.set_ylabel('Frequency [%]')
for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y),ha='center', va='bottom')

In [ ]:
pip install plotly

In [ ]:
sns.countplot(final["high_risk_applicant"], palette='CMRmap')
plt.title("high_risk_applicant",fontsize=10)
plt.show()

In [ ]:
#Scatterplot matrix
import plotly.express as px
df = final
fig = px.scatter_matrix(df,
    dimensions=["Principal_loan_amount", "Primary_applicant_age_in_years","Months_loan_taken_for"],
    color="high_risk_applicant",height=900)
fig.show()

In [ ]:
#Removing outliers
final= final[final["Principal_loan_amount"]<=160]
final= final[final["Months_loan_taken_for"]<=70]

In [ ]:
#Scatterplot matrix
import plotly.express as px
df = final
fig = px.scatter_matrix(df,
    dimensions=["Principal_loan_amount", "Primary_applicant_age_in_years","Months_loan_taken_for","Loan_history"],
    color="high_risk_applicant",height=900)
fig.show()

In [ ]:
data=final
fig = px.box(data, x="Employment_status", y="Primary_applicant_age_in_years", color="high_risk_applicant",
color_discrete_sequence=px.colors.qualitative.Dark24,
labels={col:col.replace('_', ' ') for col in data.columns},
category_orders={"Employment_status":["unemployed / unskilled - non-resident","management / self-employed / highly qualified employee / officer","unskilled - resident","skilled employee / official"]})
fig.update_layout(legend=dict(orientation="h", yanchor="bottom",
y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
data=final
fig = px.box(data, x="Employment_status", y="Primary_applicant_age_in_years", color="high_risk_applicant",
color_discrete_sequence=px.colors.qualitative.Dark24,
labels={col:col.replace('_', ' ') for col in data.columns},
category_orders={"Employment_status":["unemployed / unskilled - non-resident","management / self-employed / highly qualified employee / officer","unskilled - resident","skilled employee / official"]})
fig.update_layout(legend=dict(orientation="h", yanchor="bottom",
y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
data=final
fig = px.box(data, x="high_risk_applicant", y="Number_of_existing_loans_at_this_bank", color="high_risk_applicant",
color_discrete_sequence=px.colors.qualitative.Dark24,
labels={col:col.replace('_', ' ') for col in data.columns},
category_orders={"Number_of_existing_loans_at_this_bank":[1,2,3,4]})
fig.update_layout(legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(final, x="Number_of_existing_loans_at_this_bank", y="Loan_history", color="high_risk_applicant")
fig.show()

In [ ]:
!pip install tabplot

In [ ]:
data=final
fig = px.box(data, x="Gender", y="Principal_loan_amount", color="high_risk_applicant",
color_discrete_sequence=px.colors.qualitative.Dark24,
labels={col:col.replace('_', ' ') for col in data.columns},
category_orders={"Gender":["Male","Female"]})
fig.update_layout(legend=dict(orientation="h", yanchor="bottom",
y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
data=final
fig = px.box(data, x="Loan_history", y="Primary_applicant_age_in_years", color="high_risk_applicant",
color_discrete_sequence=px.colors.qualitative.Dark24,
labels={col:col.replace('_', ' ') for col in data.columns},
category_orders={"Loan_history":['critical/pending loans at other banks',
       'existing loans paid back duly till now',
       'delay in paying off loans in the past',
       'no loans taken/all loans paid back duly',
       'all loans at this bank paid back duly']})
fig.update_layout(legend=dict(orientation="h", yanchor="bottom",
y=1.02, xanchor="right", x=1))
fig.show()

In [ ]:
final.dtypes

In [ ]:
fig = px.parallel_categories(final, color_continuous_scale=px.colors.sequential.RdBu, color="high_risk_applicant",
dimensions=["Purpose",'Property','Other_EMI_plans','Loan_history','Gender','Marital_status','Housing','Employment_status','Has_been_employed_for_at_least','Has_been_employed_for_at_most','Telephone','Savings_account_balance','Balance_in_existing_bank_account_(lower_limit_of_bucket)','Balance_in_existing_bank_account_(upper_limit_of_bucket)'], labels={col:col.replace('_', ' ') for col in data.columns})
fig.show()
#["Purpose",'Property','Other_EMI_plans','Loan_history','Gender','Marital_status','Housing','Employment_status','Has_been_employed_for_at_least','Has_been_employed_for_at_most','Telephone','Savings_account_balance','Balance_in_existing_bank_account_(lower_limit_of_bucket)','Balance_in_existing_bank_account_(upper_limit_of_bucket)']

In [ ]:
#One hot encoding of categorical variables
df = pd.get_dummies(data=final,columns=["Purpose",'Property','Other_EMI_plans','Loan_history','Gender','Marital_status','Housing','Employment_status','Has_been_employed_for_at_least','Has_been_employed_for_at_most','Telephone','Savings_account_balance','Balance_in_existing_bank_account_(lower_limit_of_bucket)','Balance_in_existing_bank_account_(upper_limit_of_bucket)'])

In [ ]:
pip install -U scikit-learn

In [ ]:
import sklearn
print (sklearn.__version__)

In [ ]:
Y = df['high_risk_applicant']
X = df.drop('high_risk_applicant',axis=1)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, random_state=0, test_size=.20)

In [ ]:
from sklearn.metrics import classification_report
def model_assess(model, name):
  model.fit(x_train, y_train)
  preds = model.predict(x_test)
  preds_proba = model.predict_proba(x_test)
  print('                   ', name, '\n',classification_report(y_test, model.predict(x_test)))
from sklearn.neighbors import KNeighborsClassifier
#KNN
knn=KNeighborsClassifier(n_neighbors=151)
model_assess(knn,'knn')
from sklearn.linear_model import LogisticRegression
#Logistic Regression
lg = LogisticRegression(random_state=0)
model_assess(lg, 'Logistic Regression')
#XGB
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_assess(xgb, 'XGBoost')


In [ ]:

#Feature importance plot
from xgboost import plot_importance
fig, (ax1,ax2) = plt.subplots(figsize = (100, 100), ncols=1, nrows=2)
plt.subplots_adjust(left=0.125, right=0.9, bottom=0.1, top = 0.9, wspace=0, hspace = 0.5)
plot_importance(xgb, importance_type='gain', ax = ax1)
ax1.set_title('Feature Importance by Information Gain', fontsize = 18)
ax1.set_xlabel('Gain')
